In [2]:
import pandas as pd
import sys
import numpy as np

In [3]:
# set parameter
net_name = ["Lenet", "GoogLenet", "Squeezenet", "Alexnet", "Mobilenet"]
device_name = "RedmiNote9P"
fre_list = ["576000", "768000", "1017600", "1248000", "1324800", "1516800", "1612800", "1708800", "1804800"]  # Set Frequency in KHz:  没有办法降频到300000
freq_big_list=["787200", "979200", "1036800", "1248000", "1401600", "1555200", "1766400", "1900800", "2073600", "2131200", "2208000"]
# batchsizes = [1, 2, 4, 8, 16]
batchsizes = [1, 2, 3, 4, 5, 6, 7, 8]

In [16]:
# 单个freq，不同网络、不同batchsize 训练过程中消耗的能量 p*t p使用所有训练时刻的电压*电流均值（只看usb）减去基准功率（前5条数据的最小值/均值/最大值）t使用end - start ｜stamp
df = pd.DataFrame(columns=net_name, index=batchsizes)

path_to_baseline = "./" + device_name + + "/processed_data/Lenet/processed_data_Lenet.csv"
perform = pd.read_csv(path_to_baseline, index_col=0)
baseline_warmup = 10
baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值

for net in net_name:
    print(net)
    for batchsize in batchsizes:
        df[net][batchsize] = 0
        path_to_stamp = "./" + device_name + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        perform = pd.read_csv(path_to_perform, index_col=0)
        #print(perform)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                # print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
        avg_P = np.mean(P)
        print(avg_P)
        df[net][batchsize] = avg_P * (trainEnd - trainStart)/10**15
df.to_csv("./batchsize_energy_MEIZU_small.csv")
# print(df)
                

Lenet
897632967497.1428
921430549071.4286
996959087256.6666
1222844264895.0
1205607834843.158
GoogLenet
1285078179965.4546
1393646363479.138
1450791334616.1904
1580727442220.1135
1538754093744.5833
Squeezenet
1451033395440.4917
1568467450745.2703
1572913602042.1875
1657986595919.1892
1635869539970.5715
Alexnet
1686383367109.608
1345752704256.9092
1471482069145.532
1494079731189.4443
1565721796874.0789
Mobilenet
nan
nan
nan
nan
nan


In [1]:
# 测单个sample的energy
device_name = "RedmiNote9P_1804800"
df = pd.DataFrame(columns=net_name, index=batchsizes)

data = pd.read_csv("./batchsize_energy.csv", index_col=0)

# int warmUp = 2;
# int measureIterations = 32/BatchSize + 5;
# int loops = 1;
# if (NetName == "Lenet") { measureIterations = 1024/BatchSize + 100; } 
for batch in batchsizes:
    for net in net_name:
        if net == "Lenet":
            batchnum = 2 + 1024/batch + 100 + 1
        elif net == "Mobilenet":
            batchnum = 26
        else:
            batchnum = 26
        df[net][batch] = data[net][batch]/(batchnum*batch)

df.to_csv("./singleSample_energy.csv")

NameError: name 'pd' is not defined

In [7]:
# 测单个batch的energy
device_name = "RedmiNote9P_1804800"
df = pd.DataFrame(columns=net_name, index=batchsizes)

data = pd.read_csv("./batchsize_energy.csv", index_col=0)

# int warmUp = 2;
# int measureIterations = 32/BatchSize + 5;
# int loops = 1;
# if (NetName == "Lenet") { measureIterations = 1024/BatchSize + 100; } 
for batch in batchsizes:
    for net in net_name:
        if net == "Lenet":
            batchnum = 2 + 1024/batch + 100 + 1
        elif net == "Mobilenet":
            batchnum = 26
        else:
            batchnum = 26
        df[net][batch] = data[net][batch]/(batchnum)

df.to_csv("./singleBatch_energy.csv")